In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [13]:
# 參數設定
target_date = datetime(2020, 3, 7) # 爬取目標日期
root_url = 'https://www.ptt.cc'

# 爬取板塊資訊
res = requests.get('https://www.ptt.cc/bbs/hotboards.html')
soup = BeautifulSoup(res.text, 'lxml')

board_list = []
for ele in soup.select('.board'):
    board_url = root_url + ele['href']
    board_name = ele.select('.board-name')[0].text
    board_class = ele.select('.board-class')[0].text
    board_title = ele.select('.board-title')[0].text
    board_list.append([board_url, board_name, board_class, board_title])
    
df_board = pd.DataFrame(board_list, columns=['Board_Url', 'Board_Name', 'Board_Class', 'Board_Title'])


In [14]:
df_board

,Board_Url,Board_Name,Board_Class,Board_Title
0,https://www.ptt.cc/bbs/Gossiping/index.html,Gossiping,綜合,◎【八卦】天佑台灣 願疫情早日結束
1,https://www.ptt.cc/bbs/C_Chat/index.html,C_Chat,閒談,◎[希洽] 傑出的一手，人人有糖吃
2,https://www.ptt.cc/bbs/NBA/index.html,NBA,籃球,◎[NBA] R.I.P. Kobe Bryant
3,https://www.ptt.cc/bbs/nCoV2019/index.html,nCoV2019,防疫,◎COVID-19 武漢肺炎討論專板
4,https://www.ptt.cc/bbs/Stock/index.html,Stock,學術,◎[股版]發文請先詳閱版規
...,...,...,...,...
123,https://www.ptt.cc/bbs/Wanted/index.html,Wanted,徵求,◎[汪踢] 三月情人節-親親要口罩
124,https://www.ptt.cc/bbs/BB_Online/index.html,BB_Online,線上,◎[BBO] 最新消息請詳見置底
125,https://www.ptt.cc/bbs/Key_Mou_Pad/index.html,Key_Mou_Pad,生活,◎[鍵鼠] 小心肺炎病毒啊...
126,https://www.ptt.cc/bbs/hypermall/index.html,hypermall,購三,◎[賣場] 2020新年快樂


## PTT爬蟲

In [15]:
# 參數設定
target_date = datetime(2020, 3, 7) # 爬取目標日期
root_url = 'https://www.ptt.cc'

# 爬取板塊資訊
res = requests.get('https://www.ptt.cc/bbs/hotboards.html')
soup = BeautifulSoup(res.text, 'lxml')

board_list = []
for ele in soup.select('.board'):
    board_url = root_url + ele['href']
    board_name = ele.select('.board-name')[0].text
    board_class = ele.select('.board-class')[0].text
    board_title = ele.select('.board-title')[0].text
    board_list.append([board_url, board_name, board_class, board_title])
    
df_board = pd.DataFrame(board_list, columns=['Board_Url', 'Board_Name', 'Board_Class', 'Board_Title'])

# 進入板塊爬取文章資訊
df_article = pd.DataFrame()
for index, row in df_board.iterrows():
    
    # 獲取板塊內的文章資訊
    res = requests.get(row['Board_Url'])
    
    # 若需要驗證18歲，則需要使用post與session來完成
    if '我同意，我已年滿十八歲' in res.text:
        pay_load = {'from': row['Board_Url'].replace('https://www.ptt.cc/', ''),
                    'yes': 'yes'}
        rs = requests.session()
        res = rs.post('https://www.ptt.cc/ask/over18', data=pay_load)
        res = rs.get(df_board['Board_Url'][0])
    
    # 將網頁資訊丟進BeautifulSoup解析
    soup = BeautifulSoup(res.text, 'lxml')
    
    while True:
        
        # 爬取文章的內容
        article_list = []
        for ele in soup.select('.r-ent'):
            try:
                article_url = root_url + ele.select('.title a')[0]['href']
                article_author = ele.select('.meta .author')[0].text.strip()
                article_date = datetime(2018, int(ele.select('.meta .date')[0].text.strip().split('/')[0]), int(ele.select('.meta .date')[0].text.strip().split('/')[1]))
                article_title = ele.select('.title')[0].text.strip()
                article_list.append([row['Board_Class'], article_url, article_author, article_date, article_title])
            except:
                pass
        
        # 將文章內容新增到df_article，並判斷是否已經爬到指定的日期
        df_article_temp = pd.DataFrame(article_list, columns=['Board_Class', 'Article_Url', 'Article_Author', 'Article_Date', 'Article_Title'])
        if df_article_temp['Article_Date'].max() >= target_date:
            df_article = df_article.append(df_article_temp[df_article_temp['Article_Date'] == target_date])
            previous_page = root_url + soup.select('.btn.wide')[1]['href']
            res = rs.get(previous_page)
            soup = BeautifulSoup(res.text, 'lxml')
            print('正在爬取 %s' % row['Board_Class'], previous_page)
        else:
            break

    print('%s 爬取結束' % row['Board_Class'])

print('爬取結束')

# 輸出Excel檔案
df_article.to_excel('PTT.xlsx', index=False)

綜合 爬取結束
閒談 爬取結束
籃球 爬取結束
防疫 爬取結束
學術 爬取結束
棒球 爬取結束
省錢 爬取結束
遊戲 爬取結束
資訊 爬取結束
綜合 爬取結束
男女 爬取結束
車車 爬取結束
聊天 爬取結束
工作 爬取結束
Hate 爬取結束
心情 爬取結束
聊天 爬取結束
主機 爬取結束
韓劇 爬取結束
硬體 爬取結束
主機 爬取結束
網購 爬取結束
轉珠 爬取結束
房屋 爬取結束
聯誼 爬取結束
家庭 爬取結束
婚姻 爬取結束
娛樂 爬取結束
韓國 爬取結束
韓國 爬取結束
綜藝 爬取結束
台南 爬取結束
生二 爬取結束
NBA. 爬取結束
高雄 爬取結束
籃球 爬取結束
平台 爬取結束
綜合 爬取結束
抓寶 爬取結束
理財 爬取結束
線上 爬取結束
球隊 爬取結束
美容 爬取結束
日本 爬取結束
買賣 爬取結束
系統 爬取結束
玄幻 爬取結束
日本 爬取結束
台中 爬取結束
經歷 爬取結束
旅遊 爬取結束
籃球 爬取結束
資訊 爬取結束
日劇 爬取結束
美容 爬取結束
博弈 爬取結束
韓國 爬取結束
綜合 爬取結束
艦娘 爬取結束
陸劇 爬取結束
職場 爬取結束
研究 爬取結束
新竹 爬取結束
臺劇 爬取結束
車車 爬取結束
資訊 爬取結束
考試 爬取結束
美體 爬取結束
買賣 爬取結束
手遊 爬取結束
手遊 爬取結束
歐美 爬取結束
CPBL 爬取結束
布袋 爬取結束
買賣 爬取結束
美食 爬取結束
資訊 爬取結束
原創 爬取結束
線上 爬取結束
美食 爬取結束
購一 爬取結束
閒談 爬取結束
工作 爬取結束
*MLB 爬取結束
音樂 爬取結束
網球 爬取結束
狩獵 爬取結束
公職 爬取結束
CPBL 爬取結束
型月 爬取結束
交通 爬取結束
韓國 爬取結束
資訊 爬取結束
資訊 爬取結束
日本 爬取結束
結婚 爬取結束
韓國 爬取結束
國營 爬取結束
日本 爬取結束
求職 爬取結束
拼圖 爬取結束
學術 爬取結束
寵物 爬取結束
攝影 爬取結束
銀行 爬取結束
扮演 爬取結束
資訊 爬取結束
手遊 爬取結束
屬性 爬取結束
硬體 爬取結束
贈送 爬取結束
遊戲 爬取結束
求職 爬取結束
理財 爬取結束
烹飪 爬取結束
軍武 爬取結束
手遊 爬取結束
硬體 爬取結束
男女 爬取結束
買賣 爬取結束
徵求 爬取結束
線上 爬取結束
購三 爬取結束
攝影 爬取結